In [1]:
def one_qubit_readout_error_matrix(qubit_n, scale = 1):
    """
    Input: which qubit on the london device, scale (for noise scaling)
    Generates the readout error matrix
    
    organized as 
    
    P(0|0)  P(0|1)
    P(1|0)  P(1|1)
    
    """
    
    p1_0 = machine.properties().qubit_property(qubit_n, 'prob_meas1_prep0')[0]
    p0_1 = machine.properties().qubit_property(qubit_n, 'prob_meas0_prep1')[0]
    errors = [p1_0, p0_1]
   
    x, y = errors
    x = scale*x
    y = scale*y
    
    matrix = [[1-x, y], [x, 1-y]]
    
    return matrix

def column_sum_to_1(matrix): 
    """
    Makes sure that columns of a 2x2 matrix sum to 1
    """
    matrix = np.transpose(matrix)
    
    for r in range(len(matrix)):
        summ = 0
        for c in range(len(matrix[r])):
            summ += matrix[r][c]
        for c in range(len(matrix[r])): 
            matrix[r][c] = (matrix[r][c])/summ
            
    matrix = np.transpose(matrix)
    return matrix
        

def two_qubit_readout_error_matrix(mat_1, mat_2):
    """
    Input: corresponding readout error matrices of the 2 qubits
    Output: their kronecker product
    """
    P = mat_1
    Q = mat_2
    
    matrix = column_sum_to_1(np.kron(P,Q))
    return matrix

def four_qubit_readout_error_matrix(mat_1, mat_2, mat_3, mat_4):
    """
    Input: corresponding readout error matrices of the 4 qubits
    Output: their kronecker product
    """
    product = np.kron(np.kron(mat_1, mat_2), np.kron(mat_3, mat_4))
    
    matrix = column_sum_to_1(product)
    return matrix

def get_2_qubit_readout_matrix_regular(scale): 
    """
    for amplifying readout errors
    """
    
    #what should happen after all
    P_scale =  one_qubit_readout_error_matrix(1, scale)
    Q_scale = one_qubit_readout_error_matrix(0, scale)
    R_scale = two_qubit_readout_error_matrix(P_scale, Q_scale)
    
    return R_scale

def get_2_qubit_readout_matrix_after_readout_measurement(scale): 
    """
    for amplifying readout errors
    """
    #what already happened
    P_1 = one_qubit_readout_error_matrix(1)
    Q_1 = one_qubit_readout_error_matrix(0)
    R_1 = two_qubit_readout_error_matrix(P_1, Q_1)
    
    #what should happen after all
    P_scale =  one_qubit_readout_error_matrix(1, scale)
    Q_scale = one_qubit_readout_error_matrix(0, scale)
    R_scale = two_qubit_readout_error_matrix(P_scale, Q_scale)
    
    #what then needs to happen 
    """
    XR_1 = R_scale
    X = R_scale*(R_1)^-1
    """
    inverse = np.linalg.inv(R_1)
    X = np.dot(R_scale, inverse)
    
    return X

def get_4_qubit_readout_matrix_after_readout_measurement(scale): 
    """
    for amplifying readout errors
    """
    #what already happened
    P_1 = one_qubit_readout_error_matrix(3)
    Q_1 = one_qubit_readout_error_matrix(2)
    S_1 = one_qubit_readout_error_matrix(1)
    T_1 = one_qubit_readout_error_matrix(0)
    
    M_1 = four_qubit_readout_error_matrix(P_1, Q_1, S_1, T_1)
    
    #what should happen after all
    P_scale =  one_qubit_readout_error_matrix(3, scale)
    Q_scale = one_qubit_readout_error_matrix(2, scale)
    S_scale = one_qubit_readout_error_matrix(1, scale)
    T_scale = one_qubit_readout_error_matrix(0, scale)
    
    M_scale = four_qubit_readout_error_matrix(P_scale, Q_scale, S_scale, T_scale)
    
    #what then needs to happen 
    """
    XM_1 = M_scale
    X = M_scale*(M_1)^-1
    """
    inverse = np.linalg.inv(M_1)
    X = np.dot(M_scale, inverse)
    
    return X

def counts_to_array(counts, qubits = 2): 
    """
    Turning counts (dict) to array format ...for matrix multiplication
    """
    keys = []
    new_counts = {}
    bits = qubits
    
    for i in range(2**bits): 
        key = DecToBinary(i)
#         print('Hi, key is ' + key + 'and counts length is ' + str(bits))
        key = SyntaxAdjust(key, bits)
        keys.append(key)
    #print(keys)
    for key in keys:
        try:
            new_counts[key] = counts[key]
        except:
            new_counts[key] = 0
    
    ##now converting counts to array 
    array = []
    for key in new_counts.keys():
        array.append(new_counts[key])
    return array


def array_to_counts(array, shots = 1024):
    """
    Turning array into counts(dict)
    """
    #making sure the array has correct number of shots
    summ = 0
    for i in array: 
        summ+=i
    new_array = [(i/summ)*1024 for i in array]
    
    counts = {'00': float(new_array[0]), '01': float(new_array[1]), '10':float(new_array[2]), '11': float(new_array[3])}
    
    return counts

def counts_after_readout_amp(counts, scale, qubits = 2):
    """
    Input: Counts, amplification factor
    Output: modified counts
    """
    array = counts_to_array(counts, qubits)
    if(qubits == 2):
        matrix = get_2_qubit_readout_matrix_after_readout_measurement(scale)
    elif(qubits == 4):
        matrix = get_4_qubit_readout_matrix_after_readout_measurement(scale)
    else: 
        print('ERROR')
    new_array = np.dot(matrix, array)
    
    return array_to_counts(new_array)

In [ ]:
import numpy as np

counts = {'1000', }